In [ ]:
!pip install selenium opencv-python pytesseract pillow
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!apt-get update
!pip install tweepy
!pip install apscheduler


**Main Script**


In [ ]:
import time
from PIL import Image
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import tweepy
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
import pytz

#------------------------------------------------------------------------------------------------------------------------#
                                           #Edit these parameters as your need#

# Authentication keys from x api
consumer_key = "consumer_key"
consumer_secret = "consumer_secret"
access_token = "access_token"
access_token_secret = "access_token_secret"

# Tweet text
tweet_text = "This is a test tweet"

#-->google sheet url
sheet_url = 'https://docs.google.com/spreadsheets/d/1kcLgjYi75MQzJd-NhTUUAbe-FCvwHb1IRMp3LFY6p00/edit#gid=0'

#-->  Define screenshot coordinates
top_left_x, top_left_y, bottom_right_x, bottom_right_y = 110, 160, 1000, 550
#------------------------------------------------------------------------------------------------------------------------#

# Time zone for EST
est = pytz.timezone("America/New_York")

def get_time_in_region():
    # Get the timezone
    timezone = est
    # Get the current time in the specified timezone
    current_time = datetime.now(timezone)
    return current_time.strftime('%Y-%m-%d %H:%M:%S')

def capture_google_sheet_screenshot(sheet_url, screenshot_path):

    # Configure Selenium to use headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    # Set the path to the ChromeDriver
    chrome_options.binary_location = "/usr/bin/chromium-browser"
    driver = webdriver.Chrome(options=chrome_options)

    # Open the Google Sheet using Selenium
    driver.get(sheet_url)

    # Wait for the page to fully load
    time.sleep(3)

    # Capture the screenshot of the entire page
    driver.save_screenshot(screenshot_path)

    # Close the driver
    driver.quit()

def crop_screenshot(screenshot_path, cropped_image_path, top_left_x, top_left_y, bottom_right_x, bottom_right_y):
    # Open the screenshot using Pillow
    image = Image.open(screenshot_path)

    # Crop the screenshot to the specified region (top-left to bottom-right)
    cropped_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))

    # Save the cropped image
    cropped_image.save(cropped_image_path)

    # Display the cropped screenshot
    cropped_image.show()

    # Output the path to the cropped screenshot
    print(f"Cropped screenshot saved at {cropped_image_path}")

def post_tweet_with_image(consumer_key, consumer_secret, access_token, access_token_secret, image_path, tweet_text):
    # Initialize the Tweepy Client for v2 (without media upload)
    client = tweepy.Client(
        consumer_key=consumer_key,
        consumer_secret=consumer_secret,
        access_token=access_token,
        access_token_secret=access_token_secret
    )

    # Initialize the Tweepy API object for v1.1 (with media upload support)
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    # Upload media using the API object
    media = api.media_upload(image_path)

    # Create Tweet using the Client, referencing the uploaded media ID
    response = client.create_tweet(
        text=tweet_text,
        media_ids=[media.media_id]
    )

    # Print the URL of the posted Tweet
    tweet_url = f"https://twitter.com/user/status/{response.data['id']}"
    print(f"[{get_time_in_region()}] Tweet posted: {tweet_url}")

if __name__ == "__main__":


    # Google Sheet URL and file paths

    screenshot_path = "/content/sheet_screenshot_full.png"
    cropped_image_path = "/content/sheet_cropped_screenshot.png"

    # Function to capture, crop, and post tweet
    def scheduled_post():
        capture_google_sheet_screenshot(sheet_url, screenshot_path)
        crop_screenshot(screenshot_path, cropped_image_path, top_left_x, top_left_y, bottom_right_x, bottom_right_y)
        post_tweet_with_image(consumer_key, consumer_secret, access_token, access_token_secret, cropped_image_path, tweet_text)

    # Creates a default Background Scheduler
    sched = BackgroundScheduler(daemon=True)

    # Time zone for EST
    est = pytz.timezone("America/New_York")

    # Schedule posts at specific times in EST
    sched.add_job(scheduled_post, trigger="cron", hour=9, minute=40, timezone=est)
    sched.add_job(scheduled_post, trigger="cron", hour=10, minute=30, timezone=est)
    sched.add_job(scheduled_post, trigger="cron", hour=13, minute=30, timezone=est)

    # Start scheduler
    sched.start()

    # Keep the script running
    while True:
        time.sleep(60)

